# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import os

os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface'
os.environ['HF_DATASETS_CACHE'] = '/root/autodl-tmp/huggingface/datasets'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/huggingface/hub'

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成，象征着天、阳气、强盛、积极、刚健等含义。乾卦的卦辞为：“元、亨、利、贞。”这四个字意味着：元，表示开始、原始、根源；亨，表示通、顺利；利，表示有益、有利；贞，表示正、正确。

乾卦具有以下特点：

1. 阳爻处于卦象的表面，表示阳光照耀大地，万物生长，具有向上的能量。
2. 阴爻代表地，象征柔顺、安静、守旧。
3. 阳爻和阴爻相互配合，形成一种动态平衡，代表天地相互依存，相互支持。
4. 乾卦所代表的能量具有积极、刚健、充满活力的特点。

在《易经》中，乾卦具有很高的地位，它代表着天地、阳刚、力量、智慧等象征。在占卜中，乾卦通常表示事物发展顺利，具有积极向上的趋势。同时，乾卦也提醒人们要积极向上，努力进取，保持阳刚之气，以达到事业的成功。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的一个卦象，它是由于两个阳爻夹一个阴爻构成，象征着天、法律、争端、诉讼、矛盾等含义。讼卦的卦辞为：“天、裳、利、不利。”这四个字意味着：天，表示宇宙、天地、天命；裳，表示诉讼、诉讼的状貌；利，表示有益、有利；不利，表示不顺利、不利。

讼卦具有以下特点：

1. 阳爻处于卦象的表面，表示阳刚之 energy。
2. 阴爻代表阴柔、安静、守旧。
3. 阳爻和阴爻相互配合，形成一种动态平衡，代表阴阳相互依存，相互支持。
4. 讼卦所代表的能量具有争端、诉讼、矛盾等特点。

在《易经》中，讼卦的地位较低，它代表事物发展不顺利，存在争端和矛盾。然而，讼卦并不是纯粹的负面象征，它也提醒人们在处理争端和矛盾时要注意遵循法律、道德和公平原则，以求达到和谐、公正的结果。同时，讼卦也告诫人们要避免过度争斗，保持内心的平和与宽容，以免陷入无谓的纷争之中。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240313_173956"#"20240225_222843"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦中的第一卦，象征着天。它由六个阳爻组成，代表着纯粹的阳刚之气，意味着兴盛和强健。乾卦所代表的天行健，鼓励人们以天道为法，自强不息。其含义是大吉大利，表示顺利通达，但必须行正道才能永远亨通。乾卦的核心哲学是坚强、正直、克己奉公，需要谨防过于强大。在事业上，乾卦象征着一切顺利、如意，但也需要时刻警惕，保持谦逊和冷静；在经商上，表示大好的发展机会，但需要冷静理性对待，不可急躁；在婚恋方面，阳刚之气盛而阴柔之气衰，需要均衡相济才能取得美满的结果。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦中的第一卦，象征着天。它由六个阳爻组成，代表着纯粹的阳刚之气，意味着兴盛和强健。乾卦所代表的天行健，鼓励人们以天道为法，自强不息。其含义是大吉大利，表示顺利通达，但必须行正道才能永远亨通。乾卦的核心哲学是坚强、正直、克己奉公，需要谨防过于强大。在事业上，乾卦象征着一切顺利、如意，但也需要时刻警惕，保持谦逊和冷静；在经商上，表示大好的发展机会，但需要冷静理性对待，不可急躁；在婚恋方面，阳刚之气盛而阴柔之气衰，需要均衡相济才能取得美满的结果。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦是一个充满警惕和挑战的卦象。它由乾卦（天）与坎卦（水）相叠而成，代表着天和水背道而驰，形成争讼的局面。虽然中间可能有利可图，却极易陷入终凶的境地。讼卦提醒人们，在谋事之初必须审慎小心，杜绝争讼为要，避免陷入是非之争。', 'content': '\n讼卦隐含的核心哲学是：即使事情起初看似顺利，但随后可能陷入凶险。因此，不应固执已见，而是应该在适当时候退让解争。这样做，可以避免意外之灾，安于正理。同时，讼卦也提醒着人们，不要贪图不义之财，应坚持公正、公平、互利的原则，这样才能化解冲突，取得良好的结果。在决策上，应当遵循现实情况，知足适可而止，避免好胜而得罪他人，带来诉讼之灾。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个充满警惕和挑战的卦象。它由乾卦（天）与坎卦（水）相叠而成，代表着天和水背道而驰，形成争讼的局面。虽然中间可能有利可图，却极易陷入终凶的境地。讼卦提醒人们，在谋事之初必须审慎小心，杜绝争讼为要，避免陷入是非之争。

讼卦隐含的核心哲学是：即使事情起初看似顺利，但随后可能陷入凶险。因此，不应固执已见，而是应该在适当时候退让解争。这样做，可以避免意外之灾，安于正理。同时，讼卦也提醒着人们，不要贪图不义之财，应坚持公正、公平、互利的原则，这样才能化解冲突，取得良好的结果。在决策上，应当遵循现实情况，知足适可而止，避免好胜而得罪他人，带来诉讼之灾。


In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个充满深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

content:"师卦解释"
summary:"师卦代表总指挥的军情，象征着兵力和农力的结合。在《象辞》中，“地中有水”的卦象意味着君子应取法于大地容纳江河，收容和畜养大众。师卦的解释中，只有德高望重的长者统率军队才能吉祥无咎。这一卦象暗示着时运中包容别人，修行待时的时机，必须有财有库并珍惜，家宅方面有喜庆的联姻，健康方面需注意调气无忧。在传统解卦中，师卦意味着养兵聚众，出师攻伐的象征，需要正规行事与密切合作，忌独断独行和投机取巧。在事业、经商、求名和婚恋方面都需要谨慎小心，审时度势，严格要求自己，力求中正老成，方能取得成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

content:"师卦解释"
summary:"师卦代表总指挥的军情，象征着兵力和农力的结合。在《象辞》中

In [15]:
base_response, ft_response = compare_chatglm_results("周易中的谦卦有什么警示？", base_model, qlora_model, training_tag)

问题：周易中的谦卦有什么警示？

原始输出：
在周易中，谦卦是一个充满智慧的卦象。它由两个异卦相叠组成：下卦艮（山）和上卦坤（地）。这一卦象代表着谦逊，正如卦辞所言：地中有山，外表低下而内心高远。君子观此卦象，体悟到应当谦让和能够以公平的态度对待他人和事物。谦卦的核心哲学是：谦虚、谦让、裁取多余、增益缺乏、公平施予。

谦卦预示着通泰，预示着君子将会有所成就。在北宋易学家邵雍看来，谦卦代表着谦逊和忍让，而这种美德将带来吉利和平安。台湾国学大儒傅佩荣解释道，谦卦预示着眼前平顺、步步高升，无论是财运、家宅还是身体健康都将有所保障。

在传统解卦中，谦卦被解释为内高外低的美德。对于事业、经商、求名、婚恋等方面，谦逊、谦让、骄横之气必须被去除。谦卦预示着不断努力、保持谦虚态度的人将获得帮助，事业将会顺利发展，取得成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 周易中的谦卦有什么警示？ 在周易中，谦卦是一个充满智慧的卦象。它由两个异卦相叠组成：下卦艮（山）和上卦坤（地）。这一卦象代表着谦逊，正如卦辞所言：地中有山，外表低下而内心高远。君子观此卦象，体悟到应当谦让和能够以公平的态度对待他人和事物。谦卦的核心哲学是：谦虚、谦让、裁取多余、增益缺乏、公平施予。

谦卦预示着通泰，预示着君子将会有所成就。在北宋易学家邵雍看来，谦卦代表着谦逊和忍让，而这种美德将带来吉利和平安。台湾国学大儒傅佩荣解释道，谦卦预示着眼前平顺、步步高升，无论是财运、家宅还是身体健康都将有所保障。

在传统解卦中，谦卦被解释为内高外低的美德。对于事业、经商、求名、婚恋等方面，谦逊、谦让、骄横之气必须被去除。谦卦预示着不断努力、保持谦虚态度的人将获得帮助，事业将会顺利发展，取得成功。


In [16]:
base_response, ft_response = compare_chatglm_results("既济卦和未济卦有什么差异？", base_model, qlora_model, training_tag)

问题：既济卦和未济卦有什么差异？

原始输出：
既济卦和未济卦是周易中的两个卦象，它们都代表着亨通和成功的祥兆，但最终会带来灾祸。 both are positive signs, but they differ in their ultimate outcomes.

既济卦是由震卦（雷）上卦兑卦（泽）下卦组成的卦象，象征着救火盗，纵有疾患也不需担心。这是吉利的卦象，君子观此卦象，应当思及其道（德）如何才能通泰。

未济卦是由离卦（火）下卦坎卦（水）上卦组成的卦象，预示着水未能济通煮开，即将离散。此卦象预示着占测者诸事不利，预示着水将离散，无法相济。

尽管都是吉祥的卦象，但既济卦和未济卦在运势和行为上都有着不同的提醒。既济卦预示着终于成功，可以得出而竟程。而未济卦则预示着事未济，表示预计尚远，仍需努力。

在运势上，既济卦预示着成功并将成功，可以得出而竟程，是成功之象。而未济卦则表示尚未成功，仍需努力。在事业、经商、求名、婚恋和决策上，既济卦和未济卦都提醒着人们珍惜成果，警惕功败垂成的风险，努力完成任务，避免功败垂成。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 既济卦和未济卦有什么差异？ 既济卦是一个亨通之卦，代表着事情已经成功并取得了通达。起初吉利，但最终将会发生变故。该卦的象辞中，水在火上，暗示着水势压倒火势，预示着已经成功并取得通达的意思。在解卦过程中，需要思患预防，防微杜渐，有备无患。同时，要密切注意形势，抓住有利时机，以防物极必反。这是一个成功之象，但需要谨防盛极必衰。对事业而言，要警惕危机的随时出现，密切观察市场变化，转变经营策略。在求名、婚恋、决策等方面，也需要谨防事物出现前进停滞、变得颓废的情况。


In [17]:
base_response, ft_response = compare_chatglm_results("乾卦和坤卦分别代表什么？", base_model, qlora_model, training_tag)

问题：乾卦和坤卦分别代表什么？

原始输出：
乾卦和坤卦是八卦中的两个基本卦象，它们分别代表着天地两个不同的方面。在周易中，天地之间存在着交互作用，通过相互作用和相互影响，天地之间形成了万物。

乾卦象征着天，由六个阳爻组成，代表着积极、刚强、刚正等意义。它象征着天公无私，刚强直率，但也有盛极而衰的潜在风险。乾卦代表天，象征着一切事物都要以天为参照，保持敬畏之心，阳刚之德，顺从天命，才能得到好的结果。在运势、事业、经商、求名、婚恋等方面，都需要保持敬畏之心，积极进取，遵循天道，否则会招致灾祸。

坤卦象征着地，由六个阴爻组成，代表着柔顺、静止、内敛等意义。它象征着万物生长都依赖于天，需要以天为参照，顺从天道，保持静止、柔顺之德，才能获得成功。坤卦代表地，意味着万物生长依赖于天和地，需要保持相互关系，相互配合，保持敬畏之心，才能得到好的结果。在运势、事业、经商、求名、婚恋等方面，都需要保持敬畏之心，顺从天道，否则会招致灾祸。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240313_173956）：
[gMASK]sop 乾卦和坤卦分别代表什么？ 在周易中，乾卦和坤卦是六十四卦中的两个基本卦象，它们分别代表着天和地的特征。它们分别由六个阳爻和六个阴爻组成，象征着天地的阴阳相互渗透、相互影响。

乾卦象征着天，由六个阳爻组成，代表着纯阳、刚健、积极向上。它意味着万物标举，天批准裁剪，天批准运行，天批准裁剪。

坤卦象征着地，由六个阴爻组成，代表着柔顺、静止、消极无为。它意味着天地配合，万物裁剪，万物生长，天批准裁剪，天批准运行。

乾卦和坤卦共同构成了乾坤，是万物的基本分类，也是宇宙的基本运行方式。它们提示着人们应该阳刚阴柔，积极进取，同时也要顺应天批准裁剪，方得顺利发展。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [18]:
# from peft import PeftModel, PeftConfig

# epochs = 3
# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

# config = PeftConfig.from_pretrained(peft_model_path)
# qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
# training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [19]:
# base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

In [20]:
# base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [21]:
# base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [22]:
# from peft import PeftModel, PeftConfig

# epochs = 50
# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

# config = PeftConfig.from_pretrained(peft_model_path)
# qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
# training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [23]:
# base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

In [24]:
# base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

In [25]:
# base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)